In [1]:
import torch
print(torch.cuda.device_count())
from unsloth import FastLanguageModel

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
import transformers
from datasets import load_dataset, Dataset
import pprint
import json
from pathlib import Path
import os
import numpy as np
import time
import re
import pandas as pd

2
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
device = f"cuda"



if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")

# ------------------------------------------------------------

model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'

model_dir = f"/model/{model_id.replace('/', '-')}"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    # local_files_only=True
)
FastLanguageModel.for_inference(model)
tokenizer.padding_side = "left"

Using flash_attention_2 for attention computation.


==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sh2orc/Llama-3.1-Korean-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [38]:
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
dataset_dir = Path(f"/workspace/finetuning/dataset/{dataset_name}")

metadatas = {}
for directory in dataset_dir.iterdir():
    if directory.is_dir() and "scenario" in directory.name:
        with open(f"{directory}/metadata.json", "r", encoding="utf-8") as f:
            metadatas[directory.name] = json.loads(f.read())

print(metadatas)
        

{'scenario1': {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}}


In [42]:
file_name = "db_gt_v7.json"
file_path = f"/workspace/experiments/{file_name}"

with open(file_path, "r") as f:
    datas = json.load(f)


In [43]:
system_prompt = """
너는 질문에 대해 주어진 결과값을 참고해 정직하고 자연스러운 답변을 제공하는 인공지능 모델이다.
숫자는 소숫점 2자리까지 표시해.
만약 결과의 특정 변수가 공란/None이거나 특정 변수의 element 개수가 10개보다 많다면, 해당 변수는 건너뛰어도 무방해.
이 외의 경우에는 결과를 왜곡하지 말고 정직하게 답해.
예시를 참고해 최대한 모든 변수를 다루도록 노력해.
예시를 참고하지만 꼭 같을 필욘 없어.
"""
input_template = """
질문: {input}
Metadata: {metadata}
예시: {expectations}
결과: {result}
"""
# 결과에는 음수등 의도치 않은 결과가 포함되어 있을 수 있으니, 결과를 왜곡하지 말고 정직하게 답해주세요.

for data in datas:
    start_time = time.time()
    input, tags, scenario, result = data["Input"], data["Tags"], data["Scenario"], data["Result"]
    
    tags = [tags['Type']] if isinstance(tags, dict) else tags
    pass_tags = ["Normal", "Unknown", "Metadata", "NoData", "Unrelated"]
    pass_ = False
    for pass_tag in pass_tags:
        if pass_tag in tags:
            pass_ = True
            break
    if not pass_:
        continue
    
    print("====================================================")
    metadata = metadatas[scenario]

    variables = {}
    for instruction in result:
        i_type = instruction["type"]
        if i_type == "o":
            for k, v in instruction["returns"].items():
                type_, value = v
                # print(k, type_, value)
                if type_ == "pd":
                    # restore pd.DataFrame.to_dict(orient="index") or pd.Series.to_dict()
                    value = pd.DataFrame.from_dict(value, orient="index")
                elif type_ in ["primitive"]:
                    pass
                else:
                    raise NotImplementedError(f"Unsupported type: {k, type_, value}")
                variables[k] = value
            variables.update()

            # print(input, variables)
        elif i_type == "r":
            expectations = instruction["expectations"]
            user_input = input_template.format(
                input=input,
                expectations=expectations,
                result=variables,
                metadata=metadata
            )

            chat = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]

            print(user_input)

            inputs = tokenizer.apply_chat_template(
                    chat,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=3000,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False  # 결정론적 생성
            )
            
            # 결과 디코딩 및 파싱
            responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

            pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
            match = re.search(pattern, responses, re.DOTALL)
            result = match.group(1).strip() if match else None

            end_time = time.time()
            print("답변:")
            print(result)
            print("Time elapsed:", end_time - start_time)
            




질문: 어제 우리반과 옆반의 설정온도 차이 알려줘
예시: ['어제 우리반의 설정온도({{settemp_ours}}℃)는 옆반의 설정온도({{settemp_beside}}℃)보다 {{settemp_diff}}℃ 높습니다.']
결과: {'settemp_ours': 23.0, 'settemp_beside': 23.0, 'settemp_diff': 0.0}



답변:
어제 우리반의 설정온도(23.0℃)는 옆반의 설정온도(23.0℃)보다 0.0℃ 높습니다.
Time elapsed: 1.0374815464019775

질문: 오늘 우리반과 옆반의 평균 온도차이 알려줘
예시: ['우리반({{roomtemp_ours}}℃)이 옆반({{roomtemp_beside}}℃)보다 {{roomtemp_diff}}℃ 높습니다.']
결과: {'roomtemp_ours': 27.447997189037245, 'roomtemp_beside': 26.646521433591005, 'roomtemp_diff': 0.8014757554462406}

답변:
오늘 우리반과 옆반의 평균 온도차이는 0.8℃입니다.
Time elapsed: 0.5862863063812256

질문: 작년 겨울 우리반 평균온도 알려줘
예시: ['작년 겨울(2021-12 ~ 2022-02) 우리반의 평균 온도는 {{roomtemp_avg}}℃ 입니다.']
결과: {'roomtemp_avg': -1.0}

답변:
작년 겨울 우리반 평균온도는 -1.0℃입니다.
Time elapsed: 0.46731996536254883

질문: 올해 여름 앞반 평균온도 알려줘
예시: ['올해 여름(6월 ~ 8월) 앞반의 평균 온도는 {{roomtemp_avg}}℃ 입니다.']
결과: {'roomtemp_avg': 26.108745462794918}

답변:
올해 여름(6월 ~ 8월) 앞반의 평균 온도는 26.11℃ 입니다.
Time elapsed: 0.6861014366149902

질문: 올여름 제일 더웠던 날 알려줘
예시: ['올여름 우리반은 {{hottest_dates}}에 실내온도 {{hottest_temp}}℃로 가장 더웠습니다.']
결과: {'hottest_dates': ['2022-08-18', '2022-08-16'], 'hottest_temp': 27.5}

답변:
올여름 제일 더웠던 날은 2022년 8월 18일과 8월 16일입니다. 그 날의 실내 온도는 각각 27.5도였습